In [2]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00


In [18]:
!pip install pyxlsb

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [326]:
import pandas as pd
import numpy as np
import pymysql

from datetime import datetime, timedelta

def excel_a_fecha(valor_excel):
    # Convertir a fecha
    fecha_excel = datetime(1899, 12, 30) + timedelta(days=valor_excel)

    # Devolver la fecha
    return fecha_excel

In [327]:
ruta = "/content/drive/MyDrive/lap/RiesgoCartera/"

In [328]:
cnx = pymysql.connect(host='159.89.90.197', port=3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                           charset='utf8',db = 'volana')

**Se carga toda la cartera**

In [329]:
sql = """
select * from
cartera_distribuidor
where fecha = '2024-01-17'
"""
cursor = cnx.cursor()
cursor.execute(sql)
data_cartera = cursor.fetchall()
data_cartera = pd.DataFrame(data_cartera, columns=[i[0] for i in cursor.description])

In [330]:
data_cartera['id_distribuidor'].nunique()

17378

**Se carga la BD dada**

In [406]:
data_enviada = pd.read_excel(ruta + "/data/240115 Riesgo ponderado de la cartera actual.xlsb")

In [407]:
data_enviada.rename(columns = {"NIVEL DE RIESGO":"riesgo"}, inplace = True)
data_enviada_completa = data_enviada.query(f"not riesgo.isna()").copy()
data_enviada = data_enviada.query(f"riesgo.isna()")

In [408]:
data_enviada.shape

(530, 15)

**Se carga la última base de riesgo**

In [412]:
data_riesgo_ant = pd.read_csv(ruta + "/data/riesgo_cartera_11_30_23.csv")

In [413]:
data_riesgo_ant['id_distribuidor'].nunique()

16836

In [414]:
faltantes = data_enviada['ID DIST.']#np.setdiff1d(data_enviada['ID DIST.'].unique(), data_riesgo_ant['id_distribuidor'].unique())

In [415]:
data_cartera['capital'] = data_cartera.apply(lambda row: row['total_pagado_capital'] + row['saldo_pendiente_capital'],
                                             axis = 1)

In [416]:
np.setdiff1d(faltantes, data_cartera.id_distribuidor)

array([], dtype=int64)

In [417]:
data_cosecha = data_cartera.groupby('id_distribuidor').agg({'capital':'sum',
                                                            'saldo_pendiente_capital':'sum',
                                                            'fecha_aut_linea':'max'}).reset_index()


In [418]:
data_cosecha = data_cosecha.loc[[k in np.array(faltantes) for k in np.array(data_cosecha['id_distribuidor'])]]

In [419]:
data_cosecha['perdida_cosecha'] = 1 - (data_cosecha['capital'] - data_cosecha['saldo_pendiente_capital']) /  data_cosecha['capital']

<ipython-input-419-5cc3385a1307>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cosecha['perdida_cosecha'] = 1 - (data_cosecha['capital'] - data_cosecha['saldo_pendiente_capital']) /  data_cosecha['capital']


In [420]:
data_cosecha.fecha_aut_linea = pd.to_datetime(data_cosecha.fecha_aut_linea)

<ipython-input-420-4441e532c0e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cosecha.fecha_aut_linea = pd.to_datetime(data_cosecha.fecha_aut_linea)


In [421]:
from datetime import datetime, timedelta

def han_pasado_menos_de_cuatro_meses(fecha_dada):
    # Obtener la fecha actual
    fecha_actual = datetime.now()

    # Calcular la diferencia en meses
    diferencia_meses = (fecha_actual.year - fecha_dada.year) * 12 + fecha_actual.month - fecha_dada.month

    # Verificar si han pasado menos de 4 meses
    return diferencia_meses < 4

data_cosecha['menos_cuatro_meses'] = data_cosecha.fecha_aut_linea.apply(han_pasado_menos_de_cuatro_meses)


<ipython-input-421-815dbf4fdf18>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cosecha['menos_cuatro_meses'] = data_cosecha.fecha_aut_linea.apply(han_pasado_menos_de_cuatro_meses)


In [422]:
data_cosecha.perdida_cosecha.mean()

0.7891804849592771

In [423]:
data_cosecha = data_cosecha[['id_distribuidor', 'perdida_cosecha', 'menos_cuatro_meses']]

In [424]:
def setcut(cosecha_):
    #p = p*100
    cosecha_ *= 100

    grupo = 1
    cuts = np.array(sorted([100, 81.09, 72.06, 63.19, 54.27, 45.11, 36.10, 27.47, 18.17, 9.05, 0.00]))
    cosechas = np.array(sorted([0, 1.54, 3.4, 4.97, 7.38, 10.69, 11.87, 12.35, 12.68, 20.33, 22.69], reverse = True))

    #cuts = np.array(sorted([1, 0.9919, 0.975, 0.953, 0.9196, 0.8717, 0.7832, 0.6737, 0.5072, .2738]))*100
    #cosechas = np.array(sorted([2.31, 2.63, 3.1, 3.12, 3.94, 4.2, 4.33, 5.15, 5.71, 59.82], reverse = True))

    enter_ = False
    for cosecha in cosechas:
        if cosecha <= cosecha_:

            enter_ = True
            break

    if enter_:
        grupo = np.where(cosechas == cosecha)[0][0]

    return grupo, cuts[grupo], cosechas[grupo]

In [425]:
def setcut_prob(p, grupo_ = True):
    p = p*100
    cuts = np.array(sorted([100, 81.09, 72.06, 63.19, 54.27, 45.11, 36.10, 27.47, 18.17, 9.05, 0.00], reverse = True))
    cosechas = np.array(sorted([1.54, 3.4, 4.97, 7.38, 10.69, 11.87, 12.35, 12.68, 20.33, 22.69]))

    #cuts = np.array(sorted([1, 0.9919, 0.975, 0.953, 0.9196, 0.8717, 0.7832, 0.6737, 0.5072, .2738]))*100
    #cosechas = np.array(sorted([2.31, 2.63, 3.1, 3.12, 3.94, 4.2, 4.33, 5.15, 5.71, 59.82], reverse = True))

    for cut in cuts:
        if cut < p:
            break
    grupo = np.where(cuts == cut)[0][0]
    if grupo_:
        return grupo
    else:
        return cosechas[grupo - 1]

In [426]:
_, data_cosecha['probs'], data_cosecha['cosecha_canasta'] = zip(*data_cosecha.perdida_cosecha.apply(setcut))

In [427]:
def move_prob(val, menos_cuatro):
    if menos_cuatro:
      return np.random.randint(300, 850)/1000
    val /= 100
    if val == 0:
        val += np.random.randint(0, 200)/1000
    elif val == 1:
        val -= np.random.randint(0, 200)/1000
    else:
        val += np.random.randint(-200, 200)/1000
    return val

In [428]:
data_cosecha.probs = data_cosecha.apply(lambda row: move_prob(row['probs'], row['menos_cuatro_meses']), axis = 1)

In [429]:
data_cosecha

,id_distribuidor,perdida_cosecha,menos_cuatro_meses,probs,cosecha_canasta
7964,119716,NaN,True,0.323,20.33
12076,143242,NaN,True,0.829,20.33
15885,164818,NaN,True,0.539,20.33
16043,166116,0.829019,True,0.652,22.69
16060,166179,NaN,True,0.325,20.33
...,...,...,...,...,...
17272,174113,0.898877,True,0.569,22.69
17273,174121,0.889028,True,0.331,22.69
17274,174128,0.900533,True,0.843,22.69
17282,174191,0.898877,True,0.547,22.69


In [430]:
data_cosecha['grupo'] = data_cosecha.probs.apply(setcut_prob, grupo_ = True)

In [431]:
data_cosecha['cosecha_canasta'] = data_cosecha.probs.apply(setcut_prob, grupo_ = False)

In [432]:
data_cosecha.drop("perdida_cosecha", axis = 1, inplace=True)
data_cosecha.drop("menos_cuatro_meses", axis = 1, inplace=True)

In [433]:
data_cosecha

,id_distribuidor,probs,cosecha_canasta,grupo
7964,119716,0.323,12.35,7
12076,143242,0.829,1.54,1
15885,164818,0.539,10.69,5
16043,166116,0.652,4.97,3
16060,166179,0.325,12.35,7
...,...,...,...,...
17272,174113,0.569,7.38,4
17273,174121,0.331,12.35,7
17274,174128,0.843,1.54,1
17282,174191,0.547,7.38,4


In [434]:
#data_riesgo_ant.probs = data_riesgo_ant.probs / 100

In [435]:
data_riesgo_ant['cosecha_canasta'] = data_riesgo_ant.probs.apply(setcut_prob, grupo_ = False)

In [436]:
data_riesgo_ant.rename(columns={'ID DIST.': 'id_distribuidor'}, inplace = True)
data_riesgo_ant = data_riesgo_ant[['id_distribuidor', 'probs', 'grupo', 'cosecha_canasta']];

In [437]:
data_cosecha = data_cosecha[['id_distribuidor', 'probs', 'grupo', 'cosecha_canasta']]

In [438]:
data_final = pd.concat([data_riesgo_ant, data_cosecha])

In [439]:
data_final

,id_distribuidor,probs,grupo,cosecha_canasta
0,1,0.8019,2,3.40
1,19,0.0885,10,22.69
2,29,0.0795,10,22.69
3,38,0.1777,9,20.33
4,51,0.9870,1,1.54
...,...,...,...,...
17272,174113,0.5690,4,7.38
17273,174121,0.3310,7,12.35
17274,174128,0.8430,1,1.54
17282,174191,0.5470,4,7.38


In [440]:
# Guardando para la próxima
data_final.to_csv(ruta+"/data/riesgo_cartera_2024_01_18.csv")

**Uniendo lso grupos al archivo**

In [441]:
data_cosecha[['id_distribuidor', 'grupo']]

,id_distribuidor,grupo
7964,119716,7
12076,143242,1
15885,164818,5
16043,166116,3
16060,166179,7
...,...,...
17272,174113,4
17273,174121,7
17274,174128,1
17282,174191,4


In [442]:
for _, row in data_cosecha[['id_distribuidor', 'grupo']].iterrows():
  data_enviada.loc[data_enviada['ID DIST.'] == row['id_distribuidor'], "riesgo"] = row['grupo']

In [443]:
#Concatenar con la original
data_enviada

,SUCURSAL,ID COORDINADOR,COORDINADOR,ID ASESOR,ASESOR,ID DIST.,DISTRIBUIDOR,CAPITAL,DIAS ATRASO,CONVENIO,ESTATUS.,REESTRUCTURADO,REGION,BUCKET,riesgo
4051,TEXCOCO,33498,VICTOR HUGO SALAZAR SANCHEZ,32596,ELIZABETH PEREZ PERALTA,119716,JUANA GARCIA JIMENEZ,0.00,0,NO,DISTRIBUIDOR,NO,REGIÓN O,0 días,7.0
7090,COATZACOALCOS,35481,ISABEL MONSERRAT RAZO MENDEZ,26859,MONSERRAT DE LA CRUZ MARTINEZ ALVARADO,143242,JOSEFINO CHAVEZ PEREZ,0.00,0,NO,DISTRIBUIDOR,NO,REGIÓN F,0 días,1.0
10183,TONALA,31406,TEMPORAL-MIGUEL AGUSTIN CHAVEZ HUITRON,30869,LAURA PATRICIA CARRERA QUINTERO,164818,MARTINA NIETO RANGEL,0.00,0,NO,DISTRIBUIDOR,NO,REGIÓN O,0 días,5.0
10328,MILPA ALTA,26673,OMAR BANDA SILVA,35878,MARI JOSE TLAPANCO SANTIAGO,166116,SONIA ALDERETE RAMOS,10773.68,0,NO,DISTRIBUIDOR,NO,REGIÓN F,0 días,3.0
10345,MILPA ALTA,26673,OMAR BANDA SILVA,31037,BRAULIO GUZMAN JIMENEZ,166179,SAMUEL NUNEZ MORALES,0.00,0,NO,DISTRIBUIDOR,NO,REGIÓN F,0 días,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11525,IZTAPALAPA,28522,TEMPORAL-DUCISE LÓPEZ AGUDO,36153,JUAN MANUEL HERNANDEZ HERNANDEZ,174128,ITALIA 1,42999.48,0,NO,DISTRIBUIDOR RED,NO,REGIÓN O,0 días,1.0
11526,ORIZABA,27640,GUILLERMO HERNANDEZ PEREZ,27027,FERNANDO MANUEL VELASCO RIVERA,174121,NOGAL ORI,77999.83,0,NO,DISTRIBUIDOR RED,NO,REGIÓN F,0 días,7.0
11527,TLAHUAC,28369,CARLOS HUMBERTO SANCHEZ MORALES,33499,HECTOR CORONEL VENTURA,174191,AMAKAWE,317000.04,0,NO,DISTRIBUIDOR RED,NO,REGIÓN O,0 días,4.0
11528,NEZAHUALCOYOTL,33404,ISRAEL DE LOS SANTOS TRONCOSO,34012,JONNY ROMAN GARCIA VALENTINO,173911,RUDAS PANTITLAN,250000.11,0,NO,DISTRIBUIDOR RED,NO,REGIÓN O,0 días,5.0


In [445]:
data_final_enviada = pd.concat([data_enviada_completa, data_enviada])

In [451]:
data_final_enviada.to_csv(ruta + "/data/240118 Riesgo ponderado actualizado de la cartera actual.csv", index = False)